In [ ]:
__author__ = 'Brian Merino <brian.merino@noirlab.edu>, Vinicius Placco <vinicius.placco@noirlab.edu>'
__version__ = '20240606' # yyyymmdd; version datestamp of this notebook
__keywords__ = ['flamingos-2','gemini','browndwarf','dwarf','dragons']

# Gemini Flamingos-2 Brown Dwarf reduction using DRAGONS Python API
***
## Public archival data from f2img_tutorial - GS-2013B-Q-15 (WISE J041358.14-475039.3)
#### adapted from https://dragons.readthedocs.io/projects/f2img-drtutorial/en/v3.1.0/01_introduction.html
***

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [About the dataset](#About)
* [Downloading data for reduction](#Downloading_Data)
* [Set up the DRAGONS logger](#DRAGONS_logger)
* [Create File Lists](#File_Lists)
* [Create Master Dark](#Master_dark)
* [Create Bad Pixel Mask](#BPM)
* [Create Master Flat Field](#Master_Flat)
* [Reduce Science Images](#Reduce_Science)
* [Display stacked final image](#Display_Image)
* [Clean-up (optional)](#Clean-up)

<a class="anchor" id="goals"></a>
# Goals
Showcase how to reduce Flamingos-2 imaging data using the Gemini DRAGONS package on the Data Lab science platform using a custom DRAGONS kernel `"DRAGONS (Py3.7)"`. The steps include downloading data from the Gemini archive, setting up a DRAGONS calibration service, processing flats, darks, a bad pixel mask, and science frames, and creating a single combined stacked image.

<a class="anchor" id="summary"></a>
# Summary
DRAGONS is a Python-based astronomical data reduction platform written by the Gemini Science User Support Department. It can currently be used to reduce imaging data from Gemini instruments GMOS, NIRI, Flamingos 2, GSAOI, and GNIRS, as well as spectroscopic data in GMOS longslit mode. Linked <a href="https://dragons.readthedocs.io/en/v3.1.0/">here</a> is a general list of guides, manuals, and tutorials about the use of DRAGONS.

The DRAGONS kernel has been made available in the Data Lab environment, allowing users to access the routines without being dependent on installing the software on their local machines. 

In this notebook, we present an example of a DRAGONS Jupyter notebook that works in the Data Lab environment to reduce example Gemini South Flamingos-2 Y-band imaging data fully. This notebook will not present all of the details of the many options available to adjust or optimize the DRAGONS Flamingos-2 data reduction process; rather, it will just show one example of a standard reduction of a Flamingos-2 imaging dataset. 

The data used in this notebook example is Flamingos-2 Y band imaging from the Gemini archive of the brown dwarf W0413-4750 from the Gemini South program "<a href="https://archive.gemini.edu/programinfo/GS-2013B-Q-15">A Study of the 450K Transition from T to Y Dwarf, and of the 350K Y Dwarfs</a>", PI: S. Leggett, program ID GS-2013B-Q-15.

<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., <a href="http://dx.doi.org/10.1117/12.2057445">"The NOAO Data Laboratory: a conceptual overview"</a>, SPIE, 9149, 2014

* <a href="https://datalab.noirlab.edu/disclaimers.php">Data Lab disclaimer</a>

* DRAGONS publication: Labrie et al., <a href="https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract">"DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South"</a>, ASPC, 523, 321L 

* <a href="https://zenodo.org/record/7776065#.ZDg5qOzMLUI">DRAGONS open source software publication</a>

<a class="anchor" id="imports"></a>
# Importing Python libraries

In [ ]:
import warnings
import glob

from gempy.adlibrary import dataselect
from gempy.utils import logutils

from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce

from astropy.io import fits
from astropy.wcs import WCS
from astropy.utils.exceptions import AstropyWarning

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

warnings.simplefilter('ignore', category=AstropyWarning)

<a class="anchor" id="About"></a>
# About the dataset
This data comes from a Flamingos-2 program that observed a star and a distant galaxy field with dither on target for sky subtraction. 

The calibrations we use in this example include:

- Darks for the science frames.
- Flats, as a sequence of lamp-on and lamp-off exposures.
- Short darks to use with the flats to create a bad pixel mask.

| Observation Type | File name(s) | Purpose and Exposure (seconds) |
| :--- | :--- | :---: |
| Science | S20131121S0075-083 | Y-band, 120 s |
| Darks | S20131121S0369-375 | 2 s, short darks for BPM |
| Darks | S20131120S0115-120 <br> S20131121S0010 <br> S20131122S0012 <br> S20131122S0438-439| 120 s, for science data <br><br><br><br>|
| Flats | S20131129S0320-323 | 20 s, Lamp On, Y-band |
| Flats | S20131126S1111-116 | 20 s, Lamp Off, Y-band |

<a class="anchor" id="Downloading_Data"></a>
# Downloading the data

Downloading Y-band images from the Gemini archive to the current working directory. This step only needs to be executed once.

If you run this notebook for the first time and need to download the dataset, set the variable "download=True". The notebook will not redownload the dataset if it is set to False. This will become particularly useful if you run the notebooks more than once. 

In [ ]:
%%bash 

# create file that lists FITS files to be downloaded
echo "\
http://archive.gemini.edu/file/S20131121S0075.fits
http://archive.gemini.edu/file/S20131121S0076.fits
http://archive.gemini.edu/file/S20131121S0077.fits
http://archive.gemini.edu/file/S20131121S0078.fits
http://archive.gemini.edu/file/S20131121S0079.fits
http://archive.gemini.edu/file/S20131121S0080.fits
http://archive.gemini.edu/file/S20131121S0081.fits
http://archive.gemini.edu/file/S20131121S0082.fits
http://archive.gemini.edu/file/S20131121S0083.fits
http://archive.gemini.edu/file/S20131121S0369.fits
http://archive.gemini.edu/file/S20131121S0370.fits
http://archive.gemini.edu/file/S20131121S0371.fits
http://archive.gemini.edu/file/S20131121S0372.fits
http://archive.gemini.edu/file/S20131121S0373.fits
http://archive.gemini.edu/file/S20131121S0374.fits
http://archive.gemini.edu/file/S20131121S0375.fits
http://archive.gemini.edu/file/S20131120S0115.fits
http://archive.gemini.edu/file/S20131120S0116.fits
http://archive.gemini.edu/file/S20131120S0117.fits
http://archive.gemini.edu/file/S20131120S0118.fits
http://archive.gemini.edu/file/S20131120S0119.fits
http://archive.gemini.edu/file/S20131120S0120.fits
http://archive.gemini.edu/file/S20131121S0010.fits
http://archive.gemini.edu/file/S20131122S0012.fits
http://archive.gemini.edu/file/S20131122S0438.fits
http://archive.gemini.edu/file/S20131122S0439.fits
http://archive.gemini.edu/file/S20131129S0320.fits
http://archive.gemini.edu/file/S20131129S0321.fits
http://archive.gemini.edu/file/S20131129S0322.fits
http://archive.gemini.edu/file/S20131129S0323.fits
http://archive.gemini.edu/file/S20131126S1111.fits
http://archive.gemini.edu/file/S20131126S1112.fits
http://archive.gemini.edu/file/S20131126S1113.fits
http://archive.gemini.edu/file/S20131126S1114.fits
http://archive.gemini.edu/file/S20131126S1115.fits
http://archive.gemini.edu/file/S20131126S1116.fits\
" > f2.list

In [ ]:
%%bash

download="True"

if [ $download == "True" ]; then
    wget --no-check-certificate -N -q -i f2.list

else
    echo "Skipping download. To download the data set used in this notebook, set download=True."
fi

**Create a list of all the FITS files in the directory**

In [ ]:
all_files = glob.glob('S2013*[0-9].fits')
all_files.sort()

In [ ]:
# # Check header of one raw science image
# tmp = fits.open("S20131121S0075.fits")
# tmp[0].header

<a class="anchor" id="DRAGONS_logger"></a>
# Setting up the DRAGONS logger

DRAGONS comes with a local calibration manager that uses the same calibration association rules as the Gemini Observatory Archive. This allows reduce to make requests to a local light-weight database for matching processed calibrations when needed to reduce a dataset.

This tells the system where to put the calibration database. This database will keep track of the processed calibrations we will send to it.

In [ ]:
logutils.config(file_name='f2_data_reduction.log')
caldb = cal_service.set_local_database()
caldb.init("w")

<a class="anchor" id="File_Lists"></a>
# Create file lists

This data set contains science and calibration frames. For some programs, it could have different observed targets and exposure times depending on how you organize your raw data.

The DRAGONS data reduction pipeline does not organize the data for you. You have to do it. DRAGONS provides tools to help you with that.

The first step is to create lists that will be used in the data reduction process. For that, we use dataselect. Please refer to the [dataselect](https://dragons.readthedocs.io/projects/recipe-system-users-manual/en/stable/supptools/dataselect.html?highlight=dataselect) documentation for details regarding its usage.

In [ ]:
dark_files_120s = dataselect.select_data(
    all_files,['F2', 'DARK', 'RAW'],[],
    dataselect.expr_parser('exposure_time==120'))

dark_files_2s = dataselect.select_data(
    all_files,['F2', 'DARK', 'RAW'],[],
    dataselect.expr_parser('exposure_time==2'))

list_of_flats_Y = dataselect.select_data(
     all_files,['FLAT'],[],
     dataselect.expr_parser('filter_name=="Y"'))

list_of_science_images = dataselect.select_data(
    all_files,['F2'],[],
    dataselect.expr_parser('(observation_class=="science" and filter_name=="Y")'))

<a class="anchor" id="Master_dark"></a>
# Create master dark 

The final master dark will have the same name as the first file, S20131120S0115.fits, with the modified suffix S20131120S0115_dark.fits.

In [ ]:
reduce_darks = Reduce()
reduce_darks.files.extend(dark_files_120s)
reduce_darks.runr()

<a class="anchor" id="BPM"></a>
# Create a bad pixel mask

The Bad Pixel Mask (BPM) can be built using flat images with the lamps on and off and a set of short-exposure dark files. Here, our shortest dark files have a 2-second exposure time. Again, we use the reduce command to produce the BPMs.

It is important to note that the recipe library association is based on the nature of the first file in the input list. Since the recipe to make the BPM is located in the recipe library for flats, the first item on the list must be a flat.

For Flamingos-2, the filter wheel’s location is such that the filter choice does not interfere with the results. Here, we will use Y-band flats.

The BPM will have the name of the first flat file with the suffix _bpm.fits.

In [ ]:
reduce_bpm = Reduce()
reduce_bpm.files.extend(list_of_flats_Y)
reduce_bpm.files.extend(dark_files_2s)
reduce_bpm.recipename = 'makeProcessedBPM'
reduce_bpm.runr()

bpm_filename = reduce_bpm.output_filenames[0]

<a class="anchor" id="Master_Flat"></a>
# Create a master flat field

The F2 Y-band master flat is created from a series of lamp-on and lamp-off exposures. They should all have the same exposure time. Each flavor is stacked (averaged), then the lamp-off stack is subtracted from the lamp-on stack, and the result is normalized.

The master flat will have the name of the first flat file with the suffix _flat.fits.

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(list_of_flats_Y)
reduce_flats.uparms = [('addDQ:user_bpm', bpm_filename)]
reduce_flats.runr()

<a class="anchor" id="Reduce_Science"></a>
# Reduce the science images
This command retrieves the master dark and flat and applies them to the science data. For sky subtraction, the software analyses the sequence to establish whether this is a dither-on-target or an offset-to-sky sequence and proceeds accordingly. Finally, the sky-subtracted frames are aligned and stacked together. Sources in the frames are used for the alignment.

The final product file will have a _stack.fits suffix.

The output stack units are in electrons (header keyword BUNIT=electrons). The output stack is stored in a multi-extension FITS (MEF) file. The science signal is in the "SCI" extension, the variance is in the "VAR" extension, and the data quality plane (mask) is in the "DQ" extension.

Multiple files will be created by running this cell. Two new files will be created for each science frame. Both files will contain the name of the file, with the first containing the suffix _flatCorrected.fits and the second _skyCorrected.fits. In addition to these files, the final stacked image will share the name of the first science file with the suffix _stack.fits.

In [ ]:
reduce_target = Reduce()
reduce_target.files.extend(list_of_science_images)
reduce_target.uparms = [('addDQ:user_bpm', bpm_filename)]
reduce_target.runr()

<a class="anchor" id="Display_Image"></a>
# Display the stacked image

In [ ]:
image_file = "S20131121S0075_stack.fits"
hdu_list = fits.open(image_file)
wcs = WCS(hdu_list[1].header)
hdu_list.info()

In [ ]:
data = hdu_list[1].data

In [ ]:
image_data = fits.getdata(image_file, ext=1)
print(image_data.shape)

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.subplot(projection=wcs)
plt.imshow(image_data,cmap='bone',norm=Normalize(vmin=1, vmax=1000),origin='lower')

#These two lines will identify the brown dwarf in the stacked image.
plt.scatter(1149,1126,marker='o',facecolors='none',s=200,edgecolors='red')
plt.text(1095,1146,'W0413-4750',c='red')

#If you would like to see the full FOV, comment out the following two lines. 
plt.xlim(750,1600)
plt.ylim(750,1500)

ax = plt.gca()
ax.coords['ra'].set_ticklabel_position('l')
ax.coords['dec'].set_ticklabel_position('b')

ax.coords['ra'].set_axislabel('RA')
ax.coords['dec'].set_axislabel('DEC')

plt.xlabel('Right Ascension [hh:mm:ss]',fontsize=14,fontweight='bold')
plt.ylabel('Declination [degree]',fontsize=14,fontweight='bold')
plt.show()

<a class="anchor" id="Clean-up"></a>
# Optional: remove duplicate calibrations and remove raw data (uncomment lines before running)

In [ ]:
# %%bash

# cp S20131121S0075_stack.fits final.fits
# rm -r calibrations
# rm f2.list f2_data_reduction.log S2013*
# mv final.fits S20131121S0075_stack.fits